# 環境構築

In [ ]:
# ライブラリ導入
!apt-get -yq install fonts-ipafont-gothic
import numpy as np
import pandas as pd
import os
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Google Driveのマウント
drive.mount('/content/drive')

# 作業ディレクトリの変更
os.chdir('/content/drive/MyDrive/User')

# ユーザ辞書の取得

In [ ]:
# ユーザ辞書の設定
user_dict = pd.read_csv('user_dict_edit_ver2.csv', encoding = 'cp932')

# データプロセシング（顧客対応履歴）

In [ ]:
# CSV取得（開発メンバーは'メンバー選択'で取得してます）
cs_hist_df = pd.read_csv('cs_hist.csv', encoding = 'cp932')
incident_mgmt_df = pd.read_csv('incident_mgmt.csv', encoding = 'cp932')
implement = pd.read_csv('implement_edit.csv', encoding = 'cp932')
implement.columns = ['契約ID', '担当者1', '担当者2']

# データクレンジング関数の宣言
def remove_words(text, word_to_remove):
  return text.replace(word_to_remove, '')

# データクレンジング実行
word_to_remove = '\n'
cs_hist_df_edit = cs_hist_df.applymap(lambda x: remove_words(x, word_to_remove) if isinstance(x, str) else x).fillna('').loc[:, ['問合わせ番号', '受付内容', '契約ID']]
incident_mgmt_df_edit = incident_mgmt_df.applymap(lambda x: remove_words(x, word_to_remove) if isinstance(x, str) else x).fillna('').loc[:, ['問合せ番号']]

# 顧客対応からインシデントのみ抽出
cs_incident_df = cs_hist_df_edit[cs_hist_df_edit['問合わせ番号'].isin(incident_mgmt_df_edit['問合せ番号'])].reset_index(drop=True)

# 契約ID抽出

In [ ]:
def implement_extract(selected_member):
  global implement
  implement_copy = implement.copy()
  implement_copy = implement_copy[(implement_copy['担当者1'] == selected_member) | (implement_copy['担当者2'] == selected_member)]
  contract_id_list = list(implement_copy['契約ID'])
  print(contract_id_list)
  return contract_id_list

# ワードクラウド

In [ ]:
def word_cloud(contract_id_list):
  global cs_hist_df_edit, cs_incident_df, user_dict
  cs_hist_df_copy = cs_hist_df_edit.copy()
  cs_incident_copy = cs_incident_df.copy()
  cs_hist_df_copy = cs_hist_df_copy[cs_hist_df_copy['契約ID'].isin(contract_id_list)]
  cs_incident_copy = cs_incident_copy[cs_incident_copy['契約ID'].isin(contract_id_list)]

  # カウント関数の宣言
  def word_count(content_df):
    count_list = []
    for i in user_dict.index:
      search_word = user_dict['Term'][i]
      count_list.append(content_df['受付内容'].str.contains(search_word).sum())
    user_dict_count = user_dict.copy()
    user_dict_count['Count'] = count_list
    user_dict_count.sort_values(by = 'Count', ascending = False, inplace=True)
    user_dict_count.reset_index(drop = True, inplace = True)
    return user_dict_count

  cs_word_cloud_df = word_count(cs_hist_df_copy)
  incident_word_cloud_df = word_count(cs_incident_copy)

  def remove_zero(df):
    df.drop(df[df['Count'] == 0].index, inplace=True)
    return df

  cs_word_cloud_df = remove_zero(cs_word_cloud_df)
  incident_word_cloud_df = remove_zero(incident_word_cloud_df)

  # 比率計算関数の宣言
  def calc_rate(pop_df, samp_df):
    calc_df = samp_df.sort_values(by = 'Term', ascending = False).reset_index(drop = True)
    rate_df = pop_df[pop_df['Term'].isin(calc_df['Term'])].sort_values(by = 'Term', ascending = False).reset_index(drop = True)
    rate_df.columns = ['Term', 'CS Count']
    rate_df['Incident Count'] = calc_df['Count']
    rate_df['Rate'] = rate_df['Incident Count'] / rate_df['CS Count'] * 100
    rate_df['Rate'] = rate_df['Rate'].round(2)
    rate_df.sort_values(['Rate', 'CS Count'], ascending = False, inplace = True)
    rate_df.reset_index(drop = True, inplace = True)
    return rate_df

  cs_incident_rate_df = calc_rate(cs_word_cloud_df, incident_word_cloud_df)

  # ワードクラウド作成関数の宣言
  def word_cloud(word_cloud_df, freq):
    # ワードクラウド用の辞書を作成
    word_freq = dict(zip(word_cloud_df['Term'], word_cloud_df[freq]))
    # ワードクラウドを作成
    wordcloud = WordCloud(
      background_color = "white",
      width = 800,
      height = 800,
      font_path = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf',
      colormap = 'viridis',
      max_words = 50,
      ).generate_from_frequencies(word_freq)
    return wordcloud

  try:
    # ワードクラウド作成関数の実行
    word_cloud_cs = word_cloud(cs_word_cloud_df, 'Count')
    word_cloud_incident = word_cloud(incident_word_cloud_df, 'Count')
    word_cloud_rate = word_cloud(cs_incident_rate_df, 'Rate')

    # モデルの並列表示
    fig, axs = plt.subplots(1, 3, figsize=(10, 5))
    jp_font = fm.FontProperties(fname = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf')

    # 各サブプロットにデータをプロット
    axs[0].imshow(word_cloud_cs, interpolation = 'bilinear')
    axs[0].set_title('顧客対応履歴', fontproperties = jp_font)
    axs[0].axis('off')

    axs[1].imshow(word_cloud_incident, interpolation = 'bilinear')
    axs[1].set_title('インシデント', fontproperties = jp_font)
    axs[1].axis('off')

    axs[2].imshow(word_cloud_rate, interpolation = 'bilinear')
    axs[2].set_title('インシデント率', fontproperties = jp_font)
    axs[2].axis('off')

    # モデルの表示
    plt.tight_layout()
    plt.show()

    # データフレームの表示
    def display_side_by_side(*display_dfs):
      html_str = ''
      for df in display_dfs:
        df.index = np.arange(1, len(df)+1)
        html_str += df.head(50).to_html() + "\t"
      display(HTML('<div style="display: flex; justify-content: space-around;">' + html_str + '</div>'))

    display_side_by_side(cs_word_cloud_df, incident_word_cloud_df, cs_incident_rate_df)
  except:
    print('エラー：表示するデータがありません')

# メンバー選択

In [ ]:
# メンバーリストの取得
member_list = pd.read_csv('member.csv')

In [ ]:
# 選択肢のリスト
options_list = [''] + list(member_list['Name'])

# ドロップダウンウィジェットを作成
dropdown = widgets.Dropdown(
  options = options_list,
  value = options_list[0],
  description = 'メンバー：',
  disabled = False,
)

label = widgets.Label(value='選択されたメンバー：')

def execute(selected_member):
    if selected_member == '':
      return None
    else:
      display(label)
      contract_id_list = implement_extract(selected_member)
      word_cloud(contract_id_list)

def dropdown_change(change):
  if change['type'] == 'change' and change['name'] == 'value':
    new_value = change['new']
    label.value = f'選択されたメンバー： {new_value}'
    clear_output()
    execute(new_value)
    display(dropdown)

dropdown.observe(dropdown_change, names='value')

# 表示

In [ ]:
display(dropdown)